# Understanding RDD

Inspired by https://campus.datacamp.com/courses/big-data-fundamentals-with-pyspark

In [1]:
from pyspark import SparkContext
import os
from utils.stopwords import stopwords

os.environ['PYSPARK_PYTHON'] = 'python'
# sc.stop()
sc = SparkContext()

`SparkContext`

Main **entry point** for Spark functionality. A SparkContext represents the connection to a Spark cluster, and can be **used to create RDD** and broadcast variables on that cluster.

`SparkSession`

The **entry point** to programming Spark with the Dataset and DataFrame API. A SparkSession can be **used to create DataFrame**, register DataFrame as tables, execute SQL over tables, cache tables, and read parquet files.

In [2]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [3]:
print(sc)

<SparkContext master=local[*] appName=pyspark-shell>


`sc.textFile` reads a text file and return it as an RDD of Strings. Another way to create an RRD object is `.parallelize()`.

**Resilient Distributed Dataset** or **RDD** in a PySpark is a core data structure of PySpark. PySpark RDD’s is a low-level object and are highly efficient in performing distributed tasks.

PySpark RDD has a set of operations to accomplish any task. These operations are of two types:

1. Transformations

2. Actions

**Transformations** are a kind of operation that takes an RDD as input and produces another RDD as output. After applying the transformation, it creates a **Directed Acyclic Graph** or **DAG** for computations and ends after applying any actions on it. This is the reason they are called **lazy evaluation** processes.

Some popular transformations are:
- `.map`
- `.filter`

Other popular transformations: https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations


**Actions** are a kind of operation which are applied on an RDD to produce a single value. These methods are applied on a resultant RDD and produces a non-RDD value, thus removing the laziness of the transformation of RDD.

Some popular actions are:
- `.collect`
- `.take`
- `.count`

Other popular actions: https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations

Why is `.reduce` an action, but `.reduceByKey` is a transformation?

`.reduce` must pull the entire dataset down into a single location because it is reducing to one final value. `.reduceByKey` on the other hand is one value for each key. And since this action can be run on each machine locally first then it can remain an RDD and have further transformations done on its dataset.

## An example: Shakespeare's most popular words

In [4]:
file_path = "data//Complete_Shakespeare.txt"

# Create the RRD from a text file
baseRDD = sc.textFile(file_path)

# Transform to RRD of single words
splitRDD = baseRDD.flatMap(lambda x: x.split())

print("Total number of words in the RRD:", splitRDD.count())

Total number of words in the RRD: 961947


In [5]:
# Remove stop words (after converting to lower case)
splitRDD_no_stop = splitRDD.filter(lambda x: x.lower() not in stopwords)

# Create a tuple for each word containing the word as key and 1 as value
splitRDD_no_stop_words = splitRDD_no_stop.map(lambda w: (w, 1))

# Reduce by key to count the occurrences of each word
resultRDD = splitRDD_no_stop_words.reduceByKey(lambda x, y: x + y)

# Swap the keys and values so we can sort by key next
resultRDD_swap = resultRDD.map(lambda x: (x[1], x[0]))

# Sort the keys in descending order
resultRDD_swap_sort = resultRDD_swap.sortByKey(ascending=False)

for word in resultRDD_swap_sort.take(10):
	print("{},{}". format(word[1], word[0]))

thou,4512
thy,3915
shall,3247
good,2168
would,2134
Enter,2079
thee,1895
hath,1720
like,1646
you,,1581
